In [ ]:
from google.colab import drive
import geopandas as gpd
from shapely.geometry import Polygon
import folium
import matplotlib.pyplot as plt

In [ ]:
import geopandas as gpd
import os
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)  # mostra todas as linhas

# Caminho para a pasta com os arquivos .shp/.dbf/.shx/.prj
version = 7
folder = f'/content/drive/MyDrive/GEE-Samples_RMRP-v{version}'
filename = f'Samples_2015-2023_v{version}.csv'

# Lê o shapefile
df = pd.read_csv(os.path.join(folder, filename))
df = df.rename(columns={'system:index': 'feature_id'})
# Verifica as primeiras linhas
df.head()


In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import json

def add_geometry_column(df: pd.DataFrame, geo_col: str = '.geo') -> gpd.GeoDataFrame:
    """
    Converte a coluna .geo em geometria shapely e retorna um GeoDataFrame.

    Parâmetros:
        df (pd.DataFrame): DataFrame contendo a coluna .geo.
        geo_col (str): Nome da coluna com os dados GeoJSON (default: '.geo').

    Retorna:
        gpd.GeoDataFrame: GeoDataFrame com coluna 'geometry' convertida.
    """
    if geo_col not in df.columns:
        raise ValueError(f"Coluna '{geo_col}' não encontrada no DataFrame.")

    def parse_geometry(x):
        try:
            if isinstance(x, str):
                x = json.loads(x)
            return shape(x['geometry']) if 'geometry' in x else shape(x)
        except Exception as e:
            print(f"Erro ao converter geometria: {e}")
            return None

    df = df.copy()
    df['geometry'] = df[geo_col].apply(parse_geometry)

    # Remove entradas inválidas
    df = df[df['geometry'].notnull()].reset_index(drop=True)

    # Retorna como GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')
    return gdf

gdf = add_geometry_column(df)

print(gdf.geometry.geom_type.value_counts())
gdf.plot()



In [ ]:
# Verifica colunas por ano
[col for col in gdf.columns if '_2016' in col]



In [ ]:
# Dicionário de conciliação MB → classe temática
mapa_mb = {
    1: [3, 4, 5, 6, 49, 9],                # Árboreo/florestal
    2: [24],                               # Impermeabilizado
    3: [12, 15,21],                           # Vegetação herbácea
    4: [26, 33, 31],                       # Água
    5: [19, 39, 20, 40, 62, 41, 36, 46,
        47, 35, 48]                    # Agricultura
}

# Dicionário de conciliação DW → classe temática
mapa_dw = {
    1: [1],      # Árboreo/florestal
    2: [6],      # Impermeabilizado
    3: [2],      # Vegetação herbácea
    4: [0],      # Água
    5: [4]       # Agricultura
}

anos = [2016, 2017, 2018, 2019, 2020, 2021,2022, 2023]


def inverter_mapa(mapa):
    return {v: k for k, vs in mapa.items() for v in vs}

inv_mb = inverter_mapa(mapa_mb)
inv_dw = inverter_mapa(mapa_dw)

for ano in anos:
    gdf[f'mbc_{ano}'] = gdf[f'mb_{ano}'].map(inv_mb).fillna(6).astype(int)
    gdf[f'dwc_{ano}'] = gdf[f'dw_{ano}'].map(inv_dw).fillna(6).astype(int)

display(gdf[[f'mb_{ano}' for ano in anos] + [f'mbc_{ano}' for ano in anos]].head())
display(gdf[[f'dw_{ano}' for ano in anos] + [f'dwc_{ano}' for ano in anos]].head())


### Conciliação de Classes


In [ ]:
import matplotlib.pyplot as plt

# Lista de anos analisados
#anos = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2023]

# Nome das classes (pode ajustar se quiser usar texto)
nomes_classes = {
    1: "Árboreo/florestal",
    2: "Impermeabilizado",
    3: "Vegetação herbácea",
    4: "Água",
    5: "Agricultura",
    6: "Outros"
}


bandas_base = [
  'blue', 'green', 'red', 'nir', 'swir1', 'swir2',
  'ndvi', 'mndwi', 'ndbi', 'bsi', 'sub', 'veg', 'dark',
  'blue_std', 'green_std', 'red_std', 'nir_std', 'swir1_std', 'swir2_std',
  'ndvi_std', 'mndwi_std', 'ndbi_std', 'bsi_std', 'sub_std', 'veg_std', 'dark_std'
]


# Define subplots (2 linhas, n colunas)
fig, axs = plt.subplots(len(anos), 2, figsize=(12, len(anos)*3))
fig.suptitle('Distribuição por classe (mbc vs dwc)', fontsize=16)

for i, ano in enumerate(anos):
    # Contagens para mbc e dwc
    contagem_mbc = gdf[f'mbc_{ano}'].value_counts().sort_index()
    contagem_dwc = gdf[f'dwc_{ano}'].value_counts().sort_index()

    # Prepara rótulos e valores
    labels_mbc = [nomes_classes.get(k, str(k)) for k in contagem_mbc.index]
    labels_dwc = [nomes_classes.get(k, str(k)) for k in contagem_dwc.index]

    # Gráfico para mbc
    axs[i, 0].pie(contagem_mbc, labels=labels_mbc, autopct='%1.1f%%', startangle=90)
    axs[i, 0].set_title(f'mbc_{ano}')

    # Gráfico para dwc
    axs[i, 1].pie(contagem_dwc, labels=labels_dwc, autopct='%1.1f%%', startangle=90)
    axs[i, 1].set_title(f'dwc_{ano}')

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()


In [ ]:
import pandas as pd

# Inicializa dicionários para armazenar contagens
dados_mbc = []
dados_dwc = []

for ano in anos:
    # Contagens por classe (valor numérico)
    contagem_mbc = gdf[f'mbc_{ano}'].value_counts().sort_index()
    contagem_dwc = gdf[f'dwc_{ano}'].value_counts().sort_index()

    for classe in contagem_mbc.index:
        nome_classe = nomes_classes.get(classe, f'Classe {classe}')
        dados_mbc.append({'Ano': ano, 'Classe': nome_classe, 'Total': contagem_mbc[classe]})

    for classe in contagem_dwc.index:
        nome_classe = nomes_classes.get(classe, f'Classe {classe}')
        dados_dwc.append({'Ano': ano, 'Classe': nome_classe, 'Total': contagem_dwc[classe]})

# Converte para DataFrames
df_mbc = pd.DataFrame(dados_mbc).pivot(index='Ano', columns='Classe', values='Total').fillna(0).astype(int)
df_dwc = pd.DataFrame(dados_dwc).pivot(index='Ano', columns='Classe', values='Total').fillna(0).astype(int)

# Exibe
print("📊 Quantidade de amostras por classe (MBC):")
display(df_mbc)

print("📊 Quantidade de amostras por classe (DWC):")
display(df_dwc)


### Filtro de tendencia de Classe

In [ ]:
import numpy as np

for ano in anos:
    col_mbc = f'mbc_{ano}'
    col_dwc = f'dwc_{ano}'
    col_valid = f'v_y_{ano}'
    col_cc = f'cc_{ano}'  # ← nova coluna de classe conciliada

    # Etapa 1: validação por conciliação de classe
    conciliadas = gdf[col_mbc] == gdf[col_dwc]
    gdf[col_valid] = conciliadas.astype(int)

    # Nova coluna: classe conciliada ou -1
    gdf[col_cc] = np.where(conciliadas, gdf[col_mbc], -1)

    # Etapa 2: refinamento estatístico por classe
    for classe in gdf[col_cc].unique():
        if pd.isna(classe) or classe == -1: continue  # ignora valores inválidos

        # Filtra amostras válidas daquela classe no ano atual
        filtro = (gdf[col_valid] == 1) & (gdf[col_cc] == classe)
        subset = gdf[filtro]

        if subset.empty: continue  # pula se não houver dados

        # Bandas daquele ano
        bandas_ano = [f'{b}_{ano}' for b in bandas_base]

        # Média e desvio por banda para essa classe
        medias = subset[bandas_ano].mean()
        desvios = subset[bandas_ano].std()
        print('\n')
        print(f'Ano:{ano}  \nClasse:{nomes_classes[classe]}')
        print(f'Média:\n{medias}\n')
        print(f'stDesv.:\n{desvios}\n')


        # Aplica desvio diferente se for classe 4 (Água)
        fator_desvio = 1 if classe == 4 else 1.0

        # Máscara: todas bandas dentro da faixa [média ± desvio]
        dentro_faixa = ((gdf.loc[filtro, bandas_ano] >= (medias - fator_desvio * desvios)) &
                        (gdf.loc[filtro, bandas_ano] <= (medias + fator_desvio * desvios))).all(axis=1)

        # Atualiza as amostras que não passam o refinamento como inválidas
        idx_invalidas = dentro_faixa[~dentro_faixa].index
        gdf.loc[idx_invalidas, col_valid] = 0



gdf

In [ ]:
# Dicionário para armazenar contagens
contagem_agua = {}

for ano in anos:
    col_cc = f'cc_{ano}'
    col_valid = f'v_y_{ano}'

    total = ((gdf[col_cc] == 4) & (gdf[col_valid] == 1)).sum()
    contagem_agua[ano] = total

# Converte para DataFrame
df_agua = pd.DataFrame.from_dict(contagem_agua, orient='index', columns=['Total Água'])
df_agua.index.name = 'Ano'

# Exibe
display(df_agua)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Preparação dos dados
dados_abs = []
dados_pct = []

for ano in anos:
    col_class = f'mbc_{ano}'
    col_valido = f'v_y_{ano}'

    df_validas = gdf[gdf[col_valido] == 1]
    total_ano = len(df_validas)

    contagens = df_validas[col_class].value_counts().sort_index()
    proporcoes = df_validas[col_class].value_counts(normalize=True).sort_index()

    for classe in contagens.index:
        dados_abs.append({'ano': ano, 'classe': classe, 'total': contagens[classe]})
        dados_pct.append({'ano': ano, 'classe': classe, 'proporcao': proporcoes[classe]})

# Cria DataFrames pivotados
df_abs = pd.DataFrame(dados_abs)
df_pct = pd.DataFrame(dados_pct)

pivot_abs = df_abs.pivot(index='ano', columns='classe', values='total').fillna(0)
pivot_pct = df_pct.pivot(index='ano', columns='classe', values='proporcao').fillna(0)

# Ordena colunas e renomeia para nomes legíveis
pivot_abs = pivot_abs[sorted(pivot_abs.columns)]
pivot_pct = pivot_pct[sorted(pivot_pct.columns)]
pivot_abs.columns = [nomes_classes.get(c, str(c)) for c in pivot_abs.columns]
pivot_pct.columns = [nomes_classes.get(c, str(c)) for c in pivot_pct.columns]

# 🎨 Plotar os dois gráficos lado a lado
fig, axs = plt.subplots(1, 2, figsize=(16, 6))

# Gráfico de totais absolutos
pivot_abs.plot(kind='bar', stacked=True, ax=axs[0], colormap='tab20c')
axs[0].set_title('Total de amostras válidas por classe e ano')
axs[0].set_ylabel('Quantidade de amostras')
axs[0].set_xlabel('Ano')
axs[0].grid(axis='y', linestyle='--', alpha=0.5)
axs[0].legend(title='Classe', loc='upper left', bbox_to_anchor=(1, 1))

# Gráfico de proporções
pivot_pct.plot(kind='bar', stacked=True, ax=axs[1], colormap='tab20c')
axs[1].set_title('Proporção (%) de amostras válidas por classe e ano')
axs[1].set_ylabel('Proporção')
axs[1].set_xlabel('Ano')
axs[1].set_ylim(0, 1)
axs[1].grid(axis='y', linestyle='--', alpha=0.5)
axs[1].legend(title='Classe', loc='upper left', bbox_to_anchor=(1, 1))

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go


# Coleta de dados
dados_abs = []
dados_pct = []

for ano in anos:
    col_class = f'mbc_{ano}'
    col_valido = f'v_y_{ano}'

    df_validas = gdf[gdf[col_valido] == 1]
    total_ano = len(df_validas)

    contagens = df_validas[col_class].value_counts().sort_index()
    proporcoes = df_validas[col_class].value_counts(normalize=True).sort_index()

    for classe in contagens.index:
        nome = nomes_classes.get(classe, str(classe))
        dados_abs.append({'ano': ano, 'classe': nome, 'total': contagens[classe]})
        dados_pct.append({'ano': ano, 'classe': nome, 'proporcao': proporcoes[classe]})

# Criação dos DataFrames
df_abs = pd.DataFrame(dados_abs)
df_pct = pd.DataFrame(dados_pct)

# 🔷 Gráfico de valores absolutos (Plotly)
fig_abs = go.Figure()
for classe in df_abs['classe'].unique():
    dados = df_abs[df_abs['classe'] == classe]
    fig_abs.add_trace(go.Bar(
        x=dados['ano'],
        y=dados['total'],
        name=classe
    ))

fig_abs.update_layout(
    title='Total de amostras válidas por classe e ano (v_y_ano = 1)',
    barmode='stack',
    xaxis_title='Ano',
    yaxis_title='Quantidade de amostras',
    legend_title='Classe',
    template='plotly_white'
)
fig_abs.show()

# 🔷 Gráfico de proporções (Plotly)
fig_pct = go.Figure()
for classe in df_pct['classe'].unique():
    dados = df_pct[df_pct['classe'] == classe]
    fig_pct.add_trace(go.Bar(
        x=dados['ano'],
        y=dados['proporcao'],
        name=classe
    ))

fig_pct.update_layout(
    title='Proporção de amostras válidas por classe e ano (v_y_ano = 1)',
    barmode='stack',
    xaxis_title='Ano',
    yaxis_title='Proporção',
    yaxis=dict(tickformat='.0%'),
    legend_title='Classe',
    template='plotly_white'
)
fig_pct.show()


In [ ]:
# Constrói a matriz com anos nas linhas, classes nas colunas
tabela_abs = pd.pivot_table(
    df_abs,
    index='ano',
    columns='classe',
    values='total',
    aggfunc='sum',
    fill_value=0
)

# Mostra a tabela
display(tabela_abs)



In [ ]:
import matplotlib.pyplot as plt

# Define a linha da amostra (ex: índice 100)
linha = gdf.iloc[100]
display(linha.head(100))

# Cria dicionário com listas dos valores por banda ao longo dos anos
series = {}
for banda in bandas_base:
    valores = [linha[f'{banda}_{ano}'] for ano in anos]
    series[banda] = valores

# Plota o gráfico
plt.figure(figsize=(12, 6))

for banda, valores in series.items():
    plt.plot(anos, valores, label=banda)

plt.title('Evolução temporal dos índices para a amostra da linha 100')
plt.xlabel('Ano')
plt.ylabel('Valor dos índices')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


In [ ]:
import plotly.express as px
import pandas as pd

# Prepara os dados no formato longo, apenas com amostras válidas por ano
dados_plot = []
class_view = 2
for ano in anos:
    col_valido = f'v_y_{ano}'
    col_mbc = f'mbc_{ano}'
    col_dwc = f'dwc_{ano}'

    # Filtra apenas as amostras válidas naquele ano
    gdf_filtrado = gdf[(gdf[col_valido] == 1) & (gdf[col_mbc] == class_view)]

    for idx, row in gdf_filtrado.iterrows():
        for banda in bandas_base:
            dados_plot.append({
                'ano': ano,
                'banda': banda,
                'valor': row[f'{banda}_{ano}'],
                'amostra': idx,
                'mbc': nomes_classes.get(row[col_mbc], str(row[col_mbc])),
                'dwc': nomes_classes.get(row[col_dwc], str(row[col_dwc]))
            })

# Cria DataFrame longo para plotagem
df_longo = pd.DataFrame(dados_plot)

# Gráfico de dispersão por banda
fig = px.scatter(
    df_longo,
    x='ano',
    y='valor',
    facet_col='banda',
    facet_col_wrap=3,
    color='amostra',  # cor por amostra (pode trocar por 'mbc' ou 'dwc' se quiser)
    hover_data=['amostra', 'mbc', 'dwc'],
    title='Valores por banda para amostras válidas em cada ano',
    template='plotly_white'
)

fig.update_layout(
    height=800,
    showlegend=False,
    yaxis=dict(range=[-1, 1])  # Limita o eixo Y de -1 a 1
)

# Limpa os títulos dos subgráficos
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()



In [ ]:
print(len(gdf))
gdf

In [ ]:
import geopandas as gpd
import pandas as pd

#gdf = gdf.drop(columns='geometry')  # evita erro no melt

# Lista de bandas e colunas a incluir por ano
colunas_base = bandas_base + ['mbc', 'dwc', 'v_y', 'cc']

# Lista que armazenará os novos DataFrames por ano
linhas = []

for ano in anos:
    # Monta dict para renomear as colunas do ano atual
    renomear = {f'{c}_{ano}': c for c in colunas_base}

    # Filtra e renomeia
    df_ano = gdf[[f'{c}_{ano}' for c in colunas_base]].rename(columns=renomear)

    # Adiciona colunas de geometria e ano
    df_ano['ano'] = ano
    df_ano['geometry'] = gdf['geometry']
    df_ano['geo'] = gdf['.geo']
    df_ano['feature_id'] =  gdf['feature_id']



    linhas.append(df_ano)

# Junta todos os anos num único GeoDataFrame
gdf_long = pd.concat(linhas, ignore_index=True)

# Verifica
print("Formato final:", gdf_long.shape)
display(gdf_long.head())


In [ ]:
import os
# 2. Define caminho de destino
output_folder = folder
os.makedirs(output_folder, exist_ok=True)

# 3. Exporta o shapefile (todos os componentes: .shp, .shx, .dbf, .prj)
shapefile_name = f'Samples_2016-2023-v{version}-filtered'
gdf_long.to_csv(f"{output_folder}/{shapefile_name}.csv",index = False)#, driver='ESRI Shapefile')